### RAG Pipeline- Data Ingestion to Vector DB Pipeline

In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path


c:\Users\navac\OneDrive\Desktop\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents=[]
    pdf_dir=Path(pdf_directory)

    # Find all PDF Files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader=PyPDFLoader(str(pdf_file))
            documents=loader.load()

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file']=pdf_file.name
                doc.metadata['file_type']='pdf'

            all_documents.extend(documents)
            print(f"Loaded {len(documents)} pages")
        
        except Exception as e:
            print(f"Error: {e}")

    print(f"\n Total documents loaded: {len(all_documents)}")
    return all_documents

# process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 2 PDF files to process

Processing: DSA_Week1_QuestionBank.pdf
Loaded 7 pages

Processing: Navya_Resume_Faucek.pdf
Loaded 2 pages

 Total documents loaded: 9


In [4]:
all_pdf_documents

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-21T06:05:21+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-21T06:05:21+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '..\\data\\pdf\\DSA_Week1_QuestionBank.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1', 'source_file': 'DSA_Week1_QuestionBank.pdf', 'file_type': 'pdf'}, page_content='Day 1 – Arrays Basics\n1. Find the maximum element in an array\n2. Find the minimum element in an array\n3. Reverse an array in-place\n4. Rotate array by k positions\n5. Find the second largest element\n6. Compute prefix sums of an array\n7. Check if array is sorted\n8. Find frequency of each element\n9. Move all zeros to the end\n10. Find leaders in an array\n11. Find missing number in 1..n array\n12. Linear search for a target element\n13. Binary search in a sorted array\n14. Find intersection of two array

In [5]:
### Text Splitting get into chunks

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs=text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs

In [6]:
chunks=split_documents(all_pdf_documents)
chunks

Split 9 documents into 14 chunks

Example chunk:
Content: Day 1 – Arrays Basics
1. Find the maximum element in an array
2. Find the minimum element in an array
3. Reverse an array in-place
4. Rotate array by k positions
5. Find the second largest element
6. 
Metadata: {'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-21T06:05:21+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-21T06:05:21+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '..\\data\\pdf\\DSA_Week1_QuestionBank.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1', 'source_file': 'DSA_Week1_QuestionBank.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-21T06:05:21+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-21T06:05:21+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '..\\data\\pdf\\DSA_Week1_QuestionBank.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1', 'source_file': 'DSA_Week1_QuestionBank.pdf', 'file_type': 'pdf'}, page_content='Day 1 – Arrays Basics\n1. Find the maximum element in an array\n2. Find the minimum element in an array\n3. Reverse an array in-place\n4. Rotate array by k positions\n5. Find the second largest element\n6. Compute prefix sums of an array\n7. Check if array is sorted\n8. Find frequency of each element\n9. Move all zeros to the end\n10. Find leaders in an array\n11. Find missing number in 1..n array\n12. Linear search for a target element\n13. Binary search in a sorted array\n14. Find intersection of two array

### Embedding and VectorStoreDB

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str="all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name=model_name
        self.model=None
        self._load_model()

    def _load_model(self):
        """ Load the sentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model=SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading ,odel {self.model}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
            textx: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embeddind_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")

        print(f"Generating embeddins for {len(texts)} texts...")
        embeddings= self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

    def get_embedding_dimension(self)->int:
        """Get the embedding dimension of the model"""
        if not self.model:
            raise ValueError("Model not loaded")
        return self.model.get_sentence_embedding_dimension()

## initialize the embedding manager
embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\navac\OneDrive\Desktop\RAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\navac\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate develope

Model loaded successfully. Embedding dimension: 384
